In [1]:
import time
import pandas as pd
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

In [2]:
class ScrapMaps:
    
    '''
    This class is made for scrapping google maps
    
    Parameters
    ----------
    
        driver_path:str
            Path to find and load webdriver for Chrome browser
            
        sleep_time:int; default=2
            seconds to wait between each selenium execution on browser
            
    '''
    
    def __init__(self, driver_path:str, sleep_time:int=2):
        
        self.driver = webdriver.Chrome(service=Service(driver_path))
        self.sleep_time = sleep_time
        
    def SearchByUrl(self, url:str):
        
        # This function use the url to make a google maps search
        
        print(f"Seaching by url: {url}")
        self.driver.get(url)
        time.sleep(self.sleep_time)
        
    def ScrollDownResults(self, xpath_element_results:str, class_name_results:str):
        
        '''
        This function scroll down in the results section found in the left of google maps
        
        Parameters
        ----------
        
            xpath_element_results: str
                xpath path to manipulate the results section and scroll down
            
            class_name_results: str
                class name to identify the number of results thrown by google maps found inside
                the xpath_element_results
        '''
        
        print("Scrolling down...")
        n_before_scroll = len(self.driver.find_elements(By.CLASS_NAME, class_name_results))
        element_to_scroll = self.driver.find_element(By.XPATH, xpath_element_results)
        
        # Scroll down
        self.driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", element_to_scroll)
        time.sleep(self.sleep_time)
        
        n_after_scroll = len(self.driver.find_elements(By.CLASS_NAME, class_name_results))
        
        # Scrolling until there are no more results
        while n_before_scroll != n_after_scroll:
            
            n_before_scroll = int(n_after_scroll)
            
            element_to_scroll = self.driver.find_element(By.XPATH, xpath_element_results)
            self.driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", element_to_scroll)
            time.sleep(self.sleep_time)
            
            n_after_scroll = len(self.driver.find_elements(By.CLASS_NAME, class_name_results))
        
        print("No more results found")
        
    def GetResultsLocation(self, class_name:str, 
                           loc_attribute:str="href", name_attribute:str="aria-label") -> dict:
        
        '''
        This function get latitude, longitude and name of the results got by google maps
        '''
        
        # Retrieving all available results on maps (results)
        places = self.driver.find_elements(By.CLASS_NAME, class_name)
        longitude = []
        latitude = []
        name = []
        
        for place in places:
            
            # Filter in the url latitude and longitude (closed between the !3d and !4d characters)
            geographic_loc = re.findall("\!3d(-?\d+\.\d+)\!4d(-?\d+\.\d+)",
                                        place.get_attribute(loc_attribute))[0]
            
            latitude.append(geographic_loc[0])
            longitude.append(geographic_loc[1])
            name.append(place.get_attribute(name_attribute))
            
        return {"latitude":latitude, "longitude":longitude, "name":name}
            
    def GetResultsReviews(self, parent_class_name:str, 
                          class_name:str, name_attribute:str="aria-label") -> dict:
        
        # Retrieving all available results on maps (reviews)
        reviews = self.driver.find_elements(By.CLASS_NAME, parent_class_name)
        n_comments = []
        rating = []
        
        for review in reviews:
            
            check_review = review.find_elements(By.CLASS_NAME, class_name)
            
            if len(check_review) > 0:
        
                review_cleaned = check_review[0].get_attribute(name_attribute) 
                review_cleaned = re.findall("(\d\.\d|\s+\d+)", review_cleaned)
        
                rating.append(review_cleaned[0])
                n_comments.append(review_cleaned[1].strip())
        
            else:
                rating.append(np.NaN)
                n_comments.append(np.NaN)
                
        return {"comments":n_comments, "rating": rating}
    
    def Close(self):
        self.driver.close()

In [3]:
def ConvertMarks(text:str):
    
    '''
    Convert accent marks in spanish to its base form
    '''
    
    base_form_vowels = {"á":"a","é":"e","í":"i","ó":"o","ú":"u"}
    new_string = ""
    
    for character in text:
        
        if character in base_form_vowels.keys():
            new_string += base_form_vowels[character]
        else:
            new_string += character
    
    return new_string

In [4]:
# Getting Delegaciones from a web table
delegaciones_cdmx = pd.read_html("https://micodigopostal.org/ciudad-de-mexico/")
delegaciones_cdmx = delegaciones_cdmx[0].drop(labels=4).values.reshape(-1,)

In [5]:
# Parsing zip codes and metadata about zip codes from delegaciones
delegaciones_cdmx_clean = [delegacion.lower().replace(" ","-") for delegacion in delegaciones_cdmx]

# Replacing accent marks by its base form
for i, delegacion in enumerate(delegaciones_cdmx_clean):
    delegaciones_cdmx_clean[i] = ConvertMarks(delegacion).replace(".","")
    
# Creating url and retrieving from web zip codes in mexico city
cdmx_address = pd.DataFrame()
for url in delegaciones_cdmx_clean:
    web_page = f"https://micodigopostal.org/ciudad-de-mexico/{url}/"
    cdmx_address = pd.concat([cdmx_address, pd.read_html(web_page)[0]])

In [6]:
# Cleaning unnecesary rows using the pattern "adsbygoogle"
mask = map(lambda x: bool(re.findall("adsbygoogle", x)), cdmx_address["Asentamiento▼"])
mask = pd.Series(list(mask))
cdmx_address = cdmx_address[~mask.values].copy()

In [7]:
zip_code = cdmx_address["Código Postal"].values
zip_code.sort()

In [8]:
# Defining paths to scrap on maps
web_driver_path = "../../../Descargas/chromedriver_linux64/chromedriver"

xpath_element_results = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]'
class_name_results =  'hfpxzc'

parent_class_name_reviews = "UaQhfb fontBodyMedium".replace(" ", ".")
class_name_reviews = "ZkP5Je"

# Initialize scraping object
google_maps_scrapper = ScrapMaps(web_driver_path, 3)

oxxo_df = pd.DataFrame()
start_block = time.time()

for i, zp in enumerate(zip_code):
    
    start = time.time()
    query = f"oxxo {zp}"
    url = "https://www.google.com/maps/search/" + query.replace(" ", "+") + "/"
    
    google_maps_scrapper.SearchByUrl(url)
    google_maps_scrapper.ScrollDownResults(xpath_element_results, class_name_results)
    oxxo_loc = google_maps_scrapper.GetResultsLocation(class_name = class_name_results)
    oxxo_rev = google_maps_scrapper.GetResultsReviews(parent_class_name = parent_class_name_reviews, 
                                                  class_name=class_name_reviews)

    oxxo_current_zip = pd.merge(left=pd.DataFrame(oxxo_loc), right=pd.DataFrame(oxxo_rev), 
                  right_index=True, left_index=True)
    
    oxxo_current_zip["cp"] = zp
    
    oxxo_df = pd.concat([oxxo_df, oxxo_current_zip])
    
    end = time.time()
    print(f"This iter took {end - start} secs")
    
    # Hopefully google dont ban me
    if ((i + 1) % 10) == 0:
        
        end_block = time.time()
        print("\t10 iterations have been completed. Waiting 5 seconds")
        print(f"\tThis block took {end_block - start_block} secs")
        start_block = time.time()
        
        oxxo_df.to_csv("oxxo_coordinates.csv", index=False)
        
        time.sleep(5)
        
    if ((i + 1) % 100) == 0:
        print("\t100 iterations have been completed")
        
google_maps_scrapper.Close()

Seaching by url: https://www.google.com/maps/search/oxxo+01000/
Scrolling down...
No more results found
This iter took 72.29711055755615 secs
Seaching by url: https://www.google.com/maps/search/oxxo+01010/
Scrolling down...
No more results found
This iter took 40.18050479888916 secs
Seaching by url: https://www.google.com/maps/search/oxxo+01020/
Scrolling down...
No more results found
This iter took 58.6889283657074 secs
Seaching by url: https://www.google.com/maps/search/oxxo+01030/
Scrolling down...
No more results found
This iter took 69.52186560630798 secs
Seaching by url: https://www.google.com/maps/search/oxxo+01030/
Scrolling down...
No more results found
This iter took 46.42599296569824 secs
Seaching by url: https://www.google.com/maps/search/oxxo+01040/
Scrolling down...
No more results found
This iter took 68.55495476722717 secs
Seaching by url: https://www.google.com/maps/search/oxxo+01049/
Scrolling down...
No more results found
This iter took 52.60365390777588 secs
Seachin

KeyboardInterrupt: 